<a href="https://colab.research.google.com/github/Adamoni22/UMWF/blob/main/Case_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
pip install backtesting

In [34]:
import yfinance as yf
from prophet import Prophet
import pandas as pd
from backtesting import Backtest, Strategy

In [35]:
#Pobieranie danych historycznych dla Microstoft
data = yf.download("MSFT", start="2024-01-01", end="2024-06-05")
data.reset_index(inplace=True)

[*********************100%%**********************]  1 of 1 completed


In [36]:
#Tworzenie i dopasowanie modelu Prophet
df = data[['Date', 'Close']]
df.columns = ['ds', 'y']
model = Prophet(daily_seasonality=False, yearly_seasonality=True, weekly_seasonality=True)
model.fit(df)

DEBUG:cmdstanpy:input tempfile: /tmp/tmp5dai6d3w/i8uwpxns.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp5dai6d3w/qc6n9hy8.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=3071', 'data', 'file=/tmp/tmp5dai6d3w/i8uwpxns.json', 'init=/tmp/tmp5dai6d3w/qc6n9hy8.json', 'output', 'file=/tmp/tmp5dai6d3w/prophet_modelhymk2irv/prophet_model-20240610153859.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
15:38:59 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
15:38:59 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [37]:
#Prognozowanie przyszłych wartości
future = model.make_future_dataframe(periods=60, freq='D')
forecast = model.predict(future)
forecast = forecast[['ds', 'yhat']]
forecast.set_index('ds', inplace=True)

In [45]:
bt_data = data.set_index('Date')
bt_data = bt_data[['Open', 'High', 'Low', 'Close', 'Volume']]

class ProphetStrategy(Strategy):
    def init(self):
        super().init()
        self.forecast = self.I(lambda: forecast['yhat'].reindex(self.data.index, method='ffill'))

    def next(self):
        if self.data.Close[-1] > self.forecast[-1]:
            self.buy()
        elif self.data.Close[-1] < self.forecast[-1]:
            self.sell()

In [46]:
# Backtesting strategii
bt = Backtest(bt_data, ProphetStrategy, cash=10000)
output = bt.run()
bt.plot()

print(output)

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


Start                     2024-01-02 00:00:00
End                       2024-06-04 00:00:00
Duration                    154 days 00:00:00
Exposure Time [%]                   98.130841
Equity Final [$]                 11180.399841
Equity Peak [$]                  11556.099365
Return [%]                          11.803998
Buy & Hold Return [%]               12.187562
Return (Ann.) [%]                   30.054031
Volatility (Ann.) [%]               24.854636
Sharpe Ratio                         1.209192
Sortino Ratio                        2.289326
Calmar Ratio                         3.327525
Max. Drawdown [%]                   -9.031947
Avg. Drawdown [%]                    -2.50274
Max. Drawdown Duration       62 days 00:00:00
Avg. Drawdown Duration       14 days 00:00:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                       11.26608
Worst Trade [%]                      11.26608
Avg. Trade [%]                    